# This code need for creating datasets from FL-27,FL-32 and FL-47.

## Importing libraries

In [1]:
from __future__ import print_function, division
from skimage import io, transform
from random import randint
import shutil

import os, uuid, glob, warnings
import numpy as np

warnings.filterwarnings("ignore")

## Preparator for FL27

In [5]:
ORIGINAL_DATA_DIR = '../data/fl27/original'
CROPPED_DATA_DIR = '../data/fl27/images'
ORIGINAL_ANNOTATION = '../data/fl27/annotation.txt'
CROPPED_ANNOTATION = '../data/fl27/crop_annotation.txt'

TRAIN_SET = '../annotations/trainset.txt'
TEST_SET = '../annotations/testset.txt'

In [6]:
original_files = glob.glob(os.path.join(ORIGINAL_DATA_DIR, '*.jpg'))

In [7]:
print(len(original_files))

1079


In [8]:
def read_from_annotation(path):
    file = open(path, "r")
    content = file.readlines()
    new = [x.split(" ")[:-1] for x in content]
    return new

In [9]:
def prepare_labels(annotaton_path):
    arr = read_from_annotation(annotaton_path)
    labels = []
    for item in (arr):
        l = item[1]
        if l not in labels:
            labels.append(l)
        else:
            continue
    return labels
LABELS = prepare_labels(ORIGINAL_ANNOTATION)

In [10]:
print(LABELS)

['Adidas', 'Apple', 'BMW', 'Citroen', 'Cocacola', 'DHL', 'Fedex', 'Ferrari', 'Ford', 'Google', 'Heineken', 'HP', 'Intel', 'McDonalds', 'Mini', 'Nbc', 'Nike', 'Pepsi', 'Porsche', 'Puma', 'RedBull', 'Sprite', 'Starbucks', 'Texaco', 'Unicef', 'Vodafone', 'Yahoo']


In [12]:
def create_resized_crops(input_annotation, input_directory,
                        output_annotation, output_directory):
    annotation = read_from_annotation(input_annotation)
    output_annotation = open(output_annotation, 'w')
    for i, data in zip(range(len(annotation)),annotation):
        img_name = data[0]
        img_path = os.path.join(input_directory, img_name)
        image = io.imread(img_path)
        positions = data[-4:]
        x1 = int(positions[0])
        y1 = int(positions[1])
        x2 = int(positions[2])
        y2 = int(positions[3])

        if x1 > x2:
            tmp = x1
            x1 = x2
            x2 = tmp

        if y1 > y2:
            tmp = y1
            y1 = y2
            y2 = tmp
        
        
            
        target = data[1]
#         fl32 - dataset
        crop_resizer(image, target, x1,x1+x2,y1,y1+y2,output_annotation, output_directory)
# 
#         fl27 - dataset
#         crop_resizer(image, target, x1,x2,y1,y2,output_annotation, output_directory)
#         crop_resizer(image, target, x1-40,x2,y1,y2,output_annotation, output_directory)
#         crop_resizer(image, target, x1,x2+40,y1,y2,output_annotation, output_directory)
#         crop_resizer(image, target, x1,x2,y1-40,y2,output_annotation, output_directory)
#         crop_resizer(image, target, x1,x2,y1,y2+40,output_annotation, output_directory)

def crop_resizer(image,target, x1,x2,y1,y2, writer,directory):
    crop_img = image[y1:y2, x1:x2]
    file_name = str(uuid.uuid4().hex)+'.jpg'
    try:
        try:
            resized = transform.resize(crop_img,(224, 224))
            io.imsave(os.path.join(directory, file_name), resized)
            note = "{} {} \n".format(file_name, target)
            writer.write(note)
        except ValueError:
            pass
    except IndexError:
        pass
create_resized_crops(ORIGINAL_ANNOTATION, ORIGINAL_DATA_DIR, CROPPED_ANNOTATION, CROPPED_DATA_DIR)

Creating test and train sets for creating main train/val dataset

In [15]:
import random
def train_test_split(crop_ann_path,  test_size = 1000):
    train = read_from_annotation(crop_ann_path)
    test = []
    for i in range(test_size):
        randIndex = randint(0,len(train)-1)
        test.append(train[randIndex])
        del(train[randIndex])
    
    print(len(train))
    print(len(test))
    random.shuffle(train)
    
    
    trainset = open(TRAIN_SET, "w")
    testset = open(TEST_SET, "w")
    
    for item in train:
        tmp = "{} {} \n".format(item[0], item[1])
        trainset.write(tmp)
    trainset.close()

    for item in test:
        tmp = "{} {} \n".format(item[0], item[1])
        testset.write(tmp)
    testset.close()
train_test_split(CROPPED_ANNOTATION)

3536
1000
